In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [42]:
df = pd.read_csv("trainingData.csv")
df = df.drop('Unnamed: 0', axis=1)

In [14]:
df

,displacementX,displacementY,displacement,totalTravelX,totalTravelY,totalTravel,timeElapsed,meanStepDisplacementX,meanStepDisplacementY,meanStepDisplacement,...,label,meanVideoStepDisplacementX,meanVideoStepDisplacementY,meanVideoStepDisplacement,sdVideoMeanStepDisplacementX,sdVideoMeanStepDisplacementY,sdVideoMeanStepDisplacement,sdVideoMeanStepTravelX,sdVideoMeanStepTravelY,sdVideoMeanStepTravel
0,-37.353,-15.415,40.408772,40.603,24.497,47.420529,301.0,-0.124510,-0.051383,0.134696,...,0.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.65031,0.958145,1.142472
1,-27.248,-18.540,32.957322,33.082,28.292,43.529944,301.0,-0.090827,-0.061800,0.109858,...,0.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.65031,0.958145,1.142472
2,-29.438,-11.623,31.649486,38.020,26.041,46.083121,301.0,-0.098127,-0.038743,0.105498,...,0.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.65031,0.958145,1.142472
3,-39.837,-14.879,42.524948,46.421,34.359,57.753356,301.0,-0.132790,-0.049597,0.141750,...,0.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.65031,0.958145,1.142472
4,-1.616,0.055,1.616936,17.134,20.303,26.566629,301.0,-0.005387,0.000183,0.005390,...,0.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.65031,0.958145,1.142472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16064,201.000,37.000,204.377102,557.000,509.000,754.539595,300.0,0.670000,0.123333,0.681257,...,1.0,0.891999,1.019608,1.552108,0.778576,0.596633,0.623197,0.49163,0.552429,0.672781
16065,530.000,137.000,547.420314,548.000,311.000,630.099199,181.0,2.928177,0.756906,3.024422,...,1.0,0.891999,1.019608,1.552108,0.778576,0.596633,0.623197,0.49163,0.552429,0.672781
16066,2.000,140.000,140.014285,564.000,602.000,824.924239,300.0,0.006667,0.466667,0.466714,...,1.0,0.891999,1.019608,1.552108,0.778576,0.596633,0.623197,0.49163,0.552429,0.672781
16067,21.000,24.000,31.890437,21.000,24.000,31.890437,12.0,1.750000,2.000000,2.657536,...,1.0,0.891999,1.019608,1.552108,0.778576,0.596633,0.623197,0.49163,0.552429,0.672781


In [15]:
df.sample(frac=1)
trainsize = int(len(df.index)*0.8)
df_train = df.iloc[:trainsize,:]
df_test = df.iloc[trainsize:,:]

In [16]:
df_train

,displacementX,displacementY,displacement,totalTravelX,totalTravelY,totalTravel,timeElapsed,meanStepDisplacementX,meanStepDisplacementY,meanStepDisplacement,...,label,meanVideoStepDisplacementX,meanVideoStepDisplacementY,meanVideoStepDisplacement,sdVideoMeanStepDisplacementX,sdVideoMeanStepDisplacementY,sdVideoMeanStepDisplacement,sdVideoMeanStepTravelX,sdVideoMeanStepTravelY,sdVideoMeanStepTravel
0,-37.353,-15.415,40.408772,40.603,24.497,47.420529,301.0,-0.124510,-0.051383,0.134696,...,0.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.650310,0.958145,1.142472
1,-27.248,-18.540,32.957322,33.082,28.292,43.529944,301.0,-0.090827,-0.061800,0.109858,...,0.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.650310,0.958145,1.142472
2,-29.438,-11.623,31.649486,38.020,26.041,46.083121,301.0,-0.098127,-0.038743,0.105498,...,0.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.650310,0.958145,1.142472
3,-39.837,-14.879,42.524948,46.421,34.359,57.753356,301.0,-0.132790,-0.049597,0.141750,...,0.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.650310,0.958145,1.142472
4,-1.616,0.055,1.616936,17.134,20.303,26.566629,301.0,-0.005387,0.000183,0.005390,...,0.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.650310,0.958145,1.142472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12850,269.000,282.000,389.724261,269.000,282.000,389.724261,273.0,0.985348,1.032967,1.427561,...,0.0,0.903781,1.127982,1.604478,0.706748,0.554957,0.567699,0.524893,0.571984,0.719275
12851,227.000,211.000,309.919344,227.000,211.000,309.919344,216.0,1.050926,0.976852,1.434812,...,0.0,0.903781,1.127982,1.604478,0.706748,0.554957,0.567699,0.524893,0.571984,0.719275
12852,302.000,305.000,429.219058,302.000,305.000,429.219058,300.0,1.006667,1.016667,1.430730,...,0.0,0.903781,1.127982,1.604478,0.706748,0.554957,0.567699,0.524893,0.571984,0.719275
12853,289.000,303.000,418.724253,289.000,303.000,418.724253,289.0,1.000000,1.048443,1.448873,...,0.0,0.903781,1.127982,1.604478,0.706748,0.554957,0.567699,0.524893,0.571984,0.719275


In [28]:
# Normalizing data
NPTX = df_train.drop('label', axis=1).values
scaler = StandardScaler()
scaler.fit(NPTX)
NPTX=scaler.transform(NPTX)
NPTY = df_test.drop('label', axis=1).values
NPTY=scaler.transform(NPTY)

In [29]:
# Convert data to torch tensors
train_X = torch.tensor(NPTX).to(torch.float32)
train_y = torch.tensor(pd.DataFrame(df_train['label']).values).to(torch.float32)

test_X = torch.tensor(NPTY).to(torch.float32)
test_y = torch.tensor(pd.DataFrame(df_test['label']).values).to(torch.float32)

test_x=scaler.transform(test_X)


# Make datasets and dataloaders

train_dataset = TensorDataset(train_X, train_y)
test_dataset = TensorDataset(test_X, test_y)


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True) 

In [30]:
NPTX

array([[-1.42019864, -1.45462554, -1.42661118, ...,  0.21952741,
         1.76829405,  1.37539559],
       [-1.35362611, -1.47673125, -1.46999625, ...,  0.21952741,
         1.76829405,  1.37539559],
       [-1.368054  , -1.42780157, -1.47761095, ...,  0.21952741,
         1.76829405,  1.37539559],
       ...,
       [ 0.81548527,  0.81193556,  0.83718466, ..., -0.31153509,
         0.04982228, -0.15001605],
       [ 0.72984027,  0.7977879 ,  0.77608006, ..., -0.31153509,
         0.04982228, -0.15001605],
       [ 0.57831448,  0.56435152,  0.54490613, ..., -0.31153509,
         0.04982228, -0.15001605]])

In [31]:
# Model building- the fun stuff

model = nn.Sequential(
    nn.Flatten(), 
    nn.Linear(29, 5),
    nn.Linear(5, 1),
    nn.Sigmoid() # 
)

# model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.BCELoss() # Should modify to F2

In [32]:
# Some layers, such as Dropout, behave differently during training
model.train()
epochs = 20

for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        # Erase accumulated gradients
        optimizer.zero_grad()

        # Forward pass
        output = model(data)

        # Calculate loss
        loss = loss_fn(output, target)

        # Backward pass
        loss.backward()
        
        # Weight update
        optimizer.step()

    # Track loss each epoch
    print('Train Epoch: %d  Loss: %.4f' % (epoch + 1,  loss.item()))

RuntimeError: all elements of input should be between 0 and 1

In [45]:
null_columns=df[df.isnull().any()]
print(df[df.isnull().any(axis=1)][null_columns].head())

/var/folders/8f/167j3z857g1cpb6_79vckdjc0000gn/T/ipykernel_24186/1015979903.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  null_columns=df[df.isnull().any()]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [46]:
df.isnull().any()

displacementX                   False
displacementY                   False
displacement                    False
totalTravelX                    False
totalTravelY                    False
totalTravel                     False
timeElapsed                     False
meanStepDisplacementX            True
meanStepDisplacementY            True
meanStepDisplacement             True
meanStepTravelX                 False
meanStepTravelY                 False
meanStepTravel                  False
sdStepTravelX                    True
sdStepTravelY                    True
sdStepTravel                     True
shapiroDisplacementX            False
shapiroDisplacementY            False
shapiroDisplacement             False
lab                             False
label                           False
meanVideoStepDisplacementX      False
meanVideoStepDisplacementY      False
meanVideoStepDisplacement       False
sdVideoMeanStepDisplacementX    False
sdVideoMeanStepDisplacementY    False
sdVideoMeanS

In [48]:
df

,displacementX,displacementY,displacement,totalTravelX,totalTravelY,totalTravel,timeElapsed,meanStepDisplacementX,meanStepDisplacementY,meanStepDisplacement,...,label,meanVideoStepDisplacementX,meanVideoStepDisplacementY,meanVideoStepDisplacement,sdVideoMeanStepDisplacementX,sdVideoMeanStepDisplacementY,sdVideoMeanStepDisplacement,sdVideoMeanStepTravelX,sdVideoMeanStepTravelY,sdVideoMeanStepTravel
0,-37.353,-15.415,40.408772,40.603,24.497,47.420529,301.0,-0.124510,-0.051383,0.134696,...,0.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.65031,0.958145,1.142472
1,-27.248,-18.540,32.957322,33.082,28.292,43.529944,301.0,-0.090827,-0.061800,0.109858,...,0.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.65031,0.958145,1.142472
2,-29.438,-11.623,31.649486,38.020,26.041,46.083121,301.0,-0.098127,-0.038743,0.105498,...,0.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.65031,0.958145,1.142472
3,-39.837,-14.879,42.524948,46.421,34.359,57.753356,301.0,-0.132790,-0.049597,0.141750,...,0.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.65031,0.958145,1.142472
4,-1.616,0.055,1.616936,17.134,20.303,26.566629,301.0,-0.005387,0.000183,0.005390,...,0.0,0.018072,-0.297990,0.620333,0.285619,1.081449,0.977457,0.65031,0.958145,1.142472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16064,201.000,37.000,204.377102,557.000,509.000,754.539595,300.0,0.670000,0.123333,0.681257,...,1.0,0.891999,1.019608,1.552108,0.778576,0.596633,0.623197,0.49163,0.552429,0.672781
16065,530.000,137.000,547.420314,548.000,311.000,630.099199,181.0,2.928177,0.756906,3.024422,...,1.0,0.891999,1.019608,1.552108,0.778576,0.596633,0.623197,0.49163,0.552429,0.672781
16066,2.000,140.000,140.014285,564.000,602.000,824.924239,300.0,0.006667,0.466667,0.466714,...,1.0,0.891999,1.019608,1.552108,0.778576,0.596633,0.623197,0.49163,0.552429,0.672781
16067,21.000,24.000,31.890437,21.000,24.000,31.890437,12.0,1.750000,2.000000,2.657536,...,1.0,0.891999,1.019608,1.552108,0.778576,0.596633,0.623197,0.49163,0.552429,0.672781
